pca_bert_200features

In [1]:
from google.colab import drive
drive.mount('/content/drive') 

Mounted at /content/drive


In [2]:
import warnings
warnings.filterwarnings("ignore") 

In [3]:
import os
import numpy as np 
import pandas as pd
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, classification_report
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.neighbors import KNeighborsClassifier

In [4]:
import pandas as pd
path = "/content/drive/MyDrive/newpro/threeclass_problem/PCA_Bert/data.csv"
df = pd.read_csv(path)
df.head() 

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,Level 5,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,Level 5,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,Level 5,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",Level 5,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,Level 2,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [5]:
df.iloc[:,1].replace(['Level 1', 'Level 2', 'Level 3', 'Level 4', 'Level 5'],
                        [1, 2, 3, 4, 5], inplace=True)

In [6]:
df.head()

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,5,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,5,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,5,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",5,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,2,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [7]:
#define bins
bins = [0,2,4,5]
#name of the group 
group_names = ['Low', 'Moderate', 'High']
df['Level '] = pd.cut(df["Level "],bins,labels = group_names)

In [8]:
df.head(5)

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,High,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,High,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,High,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",High,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,Low,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [9]:
df['Level '] = df['Level '].map({'Low': 1, 'Moderate': 2, 'High': 3}).astype(int)

In [10]:
df.head()

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,3,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,3,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,3,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",3,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,1,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [11]:
import pandas as pd
path = "/content/drive/MyDrive/newpro/threeclass_problem/PCA_Bert/bert_withstopwordchangelevels_200_features.csv"
x = pd.read_csv(path)
x.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,190,191,192,193,194,195,196,197,198,199
0,0,-15.959655,7.610355,-4.614255,3.683309,2.789502,-1.250055,-1.805187,-10.514873,-4.086176,...,0.058667,0.295236,0.215044,-0.097426,0.254717,0.056799,-0.110118,0.294597,-0.363747,0.152232
1,1,-13.495674,4.785825,-0.513378,4.535705,-0.666486,-2.018108,-2.203631,5.270856,-0.704172,...,-0.304754,-0.314934,-0.275672,0.313357,0.104134,0.247758,-0.012774,-0.049355,-0.122302,0.200900
2,2,5.556126,-6.071359,15.245860,-6.528944,1.345797,5.089355,-4.129077,-5.236971,-3.814316,...,0.411289,0.194520,-0.124041,0.314740,-0.331834,-0.776545,-0.093170,-0.265941,-0.043186,-0.187165
3,3,-4.445179,1.293825,-4.332579,-6.459947,3.055774,8.987284,7.318152,-5.308832,-3.731958,...,-0.267484,-0.134557,0.106839,0.460344,0.301095,-0.136566,0.450354,0.116780,0.231269,0.156452
4,4,-12.149864,1.734085,-7.929423,-3.487395,3.067365,4.222421,-4.707823,2.094515,0.077916,...,0.111955,-0.279896,-0.089206,0.044569,0.367515,-0.003145,0.032331,0.483580,0.224911,-0.069965


In [12]:
x.drop(x.columns[[0]], axis=1, inplace=True)
x.head()

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,-15.959655,7.610355,-4.614255,3.683309,2.789502,-1.250055,-1.805187,-10.514873,-4.086176,0.203140,...,0.058667,0.295236,0.215044,-0.097426,0.254717,0.056799,-0.110118,0.294597,-0.363747,0.152232
1,-13.495674,4.785825,-0.513378,4.535705,-0.666486,-2.018108,-2.203631,5.270856,-0.704172,1.341339,...,-0.304754,-0.314934,-0.275672,0.313357,0.104134,0.247758,-0.012774,-0.049355,-0.122302,0.200900
2,5.556126,-6.071359,15.245860,-6.528944,1.345797,5.089355,-4.129077,-5.236971,-3.814316,-2.739572,...,0.411289,0.194520,-0.124041,0.314740,-0.331834,-0.776545,-0.093170,-0.265941,-0.043186,-0.187165
3,-4.445179,1.293825,-4.332579,-6.459947,3.055774,8.987284,7.318152,-5.308832,-3.731958,-2.567814,...,-0.267484,-0.134557,0.106839,0.460344,0.301095,-0.136566,0.450354,0.116780,0.231269,0.156452
4,-12.149864,1.734085,-7.929423,-3.487395,3.067365,4.222421,-4.707823,2.094515,0.077916,1.247958,...,0.111955,-0.279896,-0.089206,0.044569,0.367515,-0.003145,0.032331,0.483580,0.224911,-0.069965


In [13]:
y=df['Level '] 

**Train Test Split**

In [14]:
# Train-Test-Split is performed

from sklearn.model_selection import train_test_split

# Splitting the data into train and test
X_train, X_test, y_train, y_test = train_test_split(x, y, train_size=0.7, random_state=42)
X_train.shape, X_test.shape

((1220, 200), (524, 200))

In [15]:
print(y_train.value_counts(normalize=True)*100)
print("\n")
print(y_test.value_counts(normalize=True)*100)

2    45.327869
1    29.590164
3    25.081967
Name: Level , dtype: float64


2    45.229008
1    29.961832
3    24.809160
Name: Level , dtype: float64


**Model Building**

In [ ]:
def perceptron_classifier (X_train, y_train, X_test, y_test):
  p = Perceptron(random_state=150)
  p.fit(X_train, y_train)
  predictions_train = p.predict(X_train)
  predictions_test = p.predict(X_test)
  train_score = accuracy_score(predictions_train, y_train)
  #print("score on train data: ", train_score)
  test_score = f1_score(predictions_test, y_test, average='weighted' )
  print("Perceptron F1 score:", test_score)

In [ ]:
from sklearn.linear_model import Perceptron
perceptron_classifier(X_train, y_train, X_test, y_test) 

Perceptron F1 score: 0.3453367725192924


**Decision Tree**

In [16]:
clf_dt = DecisionTreeClassifier(max_depth=5, class_weight = "balanced")

In [17]:
clf_dt.fit(X_train,y_train)

DecisionTreeClassifier(class_weight='balanced', max_depth=5)

In [18]:
y_pred_train = clf_dt.predict(X_train)
y_pred_test = clf_dt.predict(X_test)

In [19]:
def classifcation_report_train_test(y_train, y_train_pred, y_test, y_test_pred):

    print('''
            =========================================
               CLASSIFICATION REPORT FOR TRAIN DATA
            =========================================
            ''')
    print(classification_report(y_train, y_train_pred, digits=4))

    print('''
            =========================================
               CLASSIFICATION REPORT FOR TEST DATA
            =========================================
            ''')
    print(classification_report(y_test, y_test_pred, digits=4))

In [20]:
scores = pd.DataFrame(columns=['Model', 'Train_Accuracy', 'Train_Recall', 'Train_Precision', 'Train_F1_Score', 
                               'Test_Accuracy', 'Test_Recall', 'Test_Precision', 'Test_F1_Score'])

def get_metrics(train_actual, train_predicted, test_actual, test_predicted, model_description, dataframe):

    train_accuracy  = accuracy_score(train_actual, train_predicted)
    train_recall    = recall_score(train_actual, train_predicted, average="weighted")
    train_precision = precision_score(train_actual, train_predicted, average="weighted")
    train_f1score   = f1_score(train_actual, train_predicted, average="weighted")
    
    test_accuracy   = accuracy_score(test_actual, test_predicted)
    test_recall     = recall_score(test_actual, test_predicted, average="weighted")
    test_precision  = precision_score(test_actual, test_predicted, average="weighted")
    test_f1score    = f1_score(test_actual, test_predicted, average="weighted")

    dataframe       = dataframe.append(pd.Series([model_description, 
                                                  train_accuracy, train_recall, train_precision, train_f1score,
                                                  test_accuracy, test_recall, test_precision, test_f1score],
                                                 index=scores.columns ), 
                                       ignore_index=True)

    return(dataframe)

In [21]:
scores = get_metrics(y_train, y_pred_train, y_test, y_pred_test, "DecisionTrees", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.5,0.5,0.54133,0.494787,0.35687,0.35687,0.396299,0.356945


**Random Forest**

In [22]:
clf_rf = RandomForestClassifier(class_weight = "balanced")

In [23]:
clf_rf.fit(X=X_train, y=y_train)

RandomForestClassifier(class_weight='balanced')

In [24]:
y_pred_train1 = clf_rf.predict(X_train)
y_pred_test1 = clf_rf.predict(X_test)

In [25]:
classifcation_report_train_test(y_train, y_pred_train1, y_test, y_pred_test1)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     1.0000    1.0000    1.0000       361
           2     1.0000    1.0000    1.0000       553
           3     1.0000    1.0000    1.0000       306

    accuracy                         1.0000      1220
   macro avg     1.0000    1.0000    1.0000      1220
weighted avg     1.0000    1.0000    1.0000      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2600    0.0828    0.1256       157
           2     0.4500    0.8734    0.5940       237
           3     0.2857    0.0308    0.0556       130

    accuracy                         0.4275       524
   macro avg     0.3319    0.3290    0.2584       524
weighted avg     0.3523    0.4275    0.3201       524



In [26]:
scores = get_metrics(y_train, y_pred_train1, y_test, y_pred_test1, "RandomForest", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.5,0.5,0.54133,0.494787,0.356870,0.356870,0.396299,0.356945
1,RandomForest,1.0,1.0,1.00000,1.000000,0.427481,0.427481,0.352315,0.320065


**AdaBoost**

In [ ]:
clf_adaboost =  AdaBoostClassifier()

In [ ]:
clf_adaboost.fit(X_train, y_train)

AdaBoostClassifier()

In [ ]:
y_pred_train2 = clf_adaboost.predict(X_train)
y_pred_test2 = clf_adaboost.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train2, y_test, y_pred_test2)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.5979    0.4737    0.5286       361
           2     0.5770    0.7523    0.6531       553
           3     0.6009    0.4183    0.4933       306

    accuracy                         0.5861      1220
   macro avg     0.5919    0.5481    0.5583      1220
weighted avg     0.5892    0.5861    0.5761      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2804    0.1911    0.2273       157
           2     0.4730    0.6287    0.5399       237
           3     0.3137    0.2462    0.2759       130

    accuracy                         0.4027       524
   macro avg     0.3557    0.3553    0.3477       524
weighted avg     0.3758    0.4027    0.3807       524



In [ ]:
scores = get_metrics(y_train, y_pred_train2, y_test, y_pred_test2, "Adaboost", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.532787,0.532787,0.587042,0.472315,0.431298,0.431298,0.382105,0.363948
1,RandomForest,1.000000,1.000000,1.000000,1.000000,0.442748,0.442748,0.383773,0.359139
2,Adaboost,0.586066,0.586066,0.589179,0.576149,0.402672,0.402672,0.375778,0.380705


**Gradient Boosting Classifier**

In [ ]:
clf_gbm = GradientBoostingClassifier()

In [ ]:
clf_gbm.fit(X=X_train, y=y_train)

GradientBoostingClassifier()

In [ ]:
y_pred_train3 = clf_gbm.predict(X_train)
y_pred_test3 = clf_gbm.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train3, y_test, y_pred_test3)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.9972    0.9834    0.9902       361
           2     0.9804    0.9964    0.9883       553
           3     0.9967    0.9837    0.9901       306

    accuracy                         0.9893      1220
   macro avg     0.9914    0.9878    0.9896      1220
weighted avg     0.9895    0.9893    0.9894      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.3273    0.2293    0.2697       157
           2     0.4415    0.6371    0.5216       237
           3     0.2222    0.1231    0.1584       130

    accuracy                         0.3874       524
   macro avg     0.3303    0.3298    0.3166       524
weighted avg     0.3529    0.3874    0.3560       524



In [ ]:
scores = get_metrics(y_train, y_pred_train3, y_test, y_pred_test3, "GBM", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.532787,0.532787,0.587042,0.472315,0.431298,0.431298,0.382105,0.363948
1,RandomForest,1.000000,1.000000,1.000000,1.000000,0.442748,0.442748,0.383773,0.359139
2,Adaboost,0.586066,0.586066,0.589179,0.576149,0.402672,0.402672,0.375778,0.380705
3,GBM,0.989344,0.989344,0.989466,0.989351,0.387405,0.387405,0.352884,0.356007


**SVM Model**

In [27]:
linear_svm = SVC(kernel='linear', C=1, class_weight = "balanced")

In [28]:
linear_svm.fit(X=X_train, y= y_train)

SVC(C=1, class_weight='balanced', kernel='linear')

In [29]:
y_pred_train4 = linear_svm.predict(X_train)
y_pred_test4 = linear_svm.predict(X_test)

In [30]:
classifcation_report_train_test(y_train, y_pred_train4, y_test, y_pred_test4)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.5508    0.6759    0.6070       361
           2     0.7333    0.5570    0.6331       553
           3     0.5714    0.6667    0.6154       306

    accuracy                         0.6197      1220
   macro avg     0.6185    0.6332    0.6185      1220
weighted avg     0.6387    0.6197    0.6209      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.3054    0.3949    0.3444       157
           2     0.4054    0.2532    0.3117       237
           3     0.2254    0.3000    0.2574       130

    accuracy                         0.3073       524
   macro avg     0.3121    0.3160    0.3045       524
weighted avg     0.3308    0.3073    0.3080       524



In [31]:
scores = get_metrics(y_train, y_pred_train4, y_test, y_pred_test4, "SVM", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.500000,0.500000,0.54133,0.494787,0.356870,0.356870,0.396299,0.356945
1,RandomForest,1.000000,1.000000,1.00000,1.000000,0.427481,0.427481,0.352315,0.320065
2,SVM,0.619672,0.619672,0.63871,0.620920,0.307252,0.307252,0.330798,0.308041


**KNN**

In [ ]:
knn = KNeighborsClassifier(n_neighbors=7)

In [ ]:
knn.fit(X=X_train, y=y_train)

KNeighborsClassifier(n_neighbors=7)

In [ ]:
y_pred_train5 = knn.predict(X_train)
y_pred_test5 = knn.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train5, y_test, y_pred_test5)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.5000    0.5152    0.5075       361
           2     0.5718    0.7342    0.6429       553
           3     0.4928    0.2222    0.3063       306

    accuracy                         0.5410      1220
   macro avg     0.5215    0.4905    0.4856      1220
weighted avg     0.5307    0.5410    0.5184      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2561    0.2675    0.2617       157
           2     0.4491    0.5401    0.4904       237
           3     0.1867    0.1077    0.1366       130

    accuracy                         0.3511       524
   macro avg     0.2973    0.3051    0.2962       524
weighted avg     0.3262    0.3511    0.3341       524



In [ ]:
scores = get_metrics(y_train, y_pred_train5, y_test, y_pred_test5, "KNN", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.532787,0.532787,0.587042,0.472315,0.431298,0.431298,0.382105,0.363948
1,RandomForest,1.000000,1.000000,1.000000,1.000000,0.442748,0.442748,0.383773,0.359139
2,Adaboost,0.586066,0.586066,0.589179,0.576149,0.402672,0.402672,0.375778,0.380705
3,GBM,0.989344,0.989344,0.989466,0.989351,0.387405,0.387405,0.352884,0.356007
4,SVM,0.642623,0.642623,0.643808,0.637082,0.364504,0.364504,0.345492,0.352483
5,KNN,0.540984,0.540984,0.530742,0.518418,0.351145,0.351145,0.326176,0.334103


**GNB**

In [ ]:
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()

In [ ]:
nb.fit(X=X_train, y=y_train)

GaussianNB()

In [ ]:
y_pred_train6 = nb.predict(X_train)
y_pred_test6 = nb.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train6, y_test, y_pred_test6)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.3739    0.6981    0.4870       361
           2     0.6591    0.4720    0.5501       553
           3     0.5200    0.2549    0.3421       306

    accuracy                         0.4844      1220
   macro avg     0.5177    0.4750    0.4597      1220
weighted avg     0.5398    0.4844    0.4792      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2957    0.5669    0.3886       157
           2     0.4834    0.3080    0.3763       237
           3     0.2917    0.1615    0.2079       130

    accuracy                         0.3492       524
   macro avg     0.3569    0.3455    0.3243       524
weighted avg     0.3796    0.3492    0.3382       524



In [ ]:
scores = get_metrics(y_train, y_pred_train6, y_test, y_pred_test6, "GNB", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.532787,0.532787,0.587042,0.472315,0.431298,0.431298,0.382105,0.363948
1,RandomForest,1.000000,1.000000,1.000000,1.000000,0.442748,0.442748,0.383773,0.359139
2,Adaboost,0.586066,0.586066,0.589179,0.576149,0.402672,0.402672,0.375778,0.380705
3,GBM,0.989344,0.989344,0.989466,0.989351,0.387405,0.387405,0.352884,0.356007
4,SVM,0.642623,0.642623,0.643808,0.637082,0.364504,0.364504,0.345492,0.352483
5,KNN,0.540984,0.540984,0.530742,0.518418,0.351145,0.351145,0.326176,0.334103
6,GNB,0.484426,0.484426,0.539812,0.479225,0.349237,0.349237,0.379608,0.338221
